In [19]:
import random
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from env import get_db_url
import wrangle
from pydataset import data

### As a customer analyst, I want to know who has spent the most money with us over their lifetime. I have monthly charges and tenure, so I think I will be able to use those two attributes as features to estimate total_charges. I need to do this within an average of \$5.00 per customer.

In [13]:
url = get_db_url('telco_churn')
df = pd.read_sql('''
SELECT customer_id, monthly_charges, total_charges
FROM customers
where contract_type_id = 3
''', url)
df = wrangle.drop_nulls(wrangle.wrangle_telco(df))

### Create split_scale.py that will contain the functions that follow. Each scaler function should create the object, fit and transform both train and test. They should return the scaler, train df scaled, test df scaled. Be sure your indices represent the original indices from train/test, as those represent the indices from the original dataframe. Be sure to set a random state where applicable for reproducibility!

In [31]:
def split_my_data(x, y, train_pct = .80):
    train_x, test_x, train_y, test_y = train_test_split(x, y, 
                                                        train_size = train_pct,
                                                        random_state = 123)
    return [train_x, train_y], [test_x, test_y]

In [4]:
def standard_scaler():
    
    
    pass

In [5]:
def scale_inverse():
    pass

In [6]:
def uniform_scaler():
    pass

In [7]:
def gaussian_scaler():
    pass


In [8]:
def min_max_scaler():
    pass

In [9]:
def iqr_robust_scaler():
    pass

In [18]:
train_test_split?

In [29]:
mpg = data('mpg')
subdf = mpg[['displ', 'cyl', 'hwy']].head(10)
x = subdf[['displ', 'cyl']]
y = subdf['hwy']
split_my_data(x,y, train_pct = .7)


([    displ  cyl
  6     2.8    6
  9     1.8    4
  4     2.0    4
  2     1.8    4
  7     3.1    6
  10    2.0    4
  3     2.0    4, 6     26
  9     25
  4     30
  2     29
  7     27
  10    28
  3     31
  Name: hwy, dtype: int64], [   displ  cyl
  5    2.8    6
  1    1.8    4
  8    1.8    4, 5    26
  1    29
  8    26
  Name: hwy, dtype: int64])